AWS DATA WORKSHOP - JUPYTER VERSION - 2017
===


In [1]:
!pip install --upgrade --user awscli

    100% |################################| 1.4MB 883kB/s ta 0:00:011
    100% |################################| 276kB 3.2MB/s eta 0:00:01
    100% |################################| 51kB 7.5MB/s eta 0:00:01
    100% |################################| 61kB 7.1MB/s eta 0:00:01
    100% |################################| 4.7MB 260kB/s eta 0:00:01
    100% |################################| 552kB 2.3MB/s eta 0:00:01
    100% |################################| 81kB 11.4MB/s ta 0:00:01
    100% |################################| 215kB 5.2MB/s eta 0:00:01
    100% |################################| 143kB 8.0MB/s eta 0:00:01
  Running setup.py install for PyYAML ... done
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Preparation
===
We will first list all the S3 bucket on your account and then test the access of the S3 source bucket external your account.

Let's first check the 

In [2]:
%env

{'ACCOUNTID': '129182954833',
 'CLICOLOR': '1',
 'GIT_PAGER': 'cat',
 'HOME': '/home/user6',
 'JPY_API_TOKEN': '4533893b53cf4357845026e4a95cd60f',
 'JPY_PARENT_PID': '131062',
 'JUPYTERHUB_ADMIN_ACCESS': '1',
 'JUPYTERHUB_API_TOKEN': '4533893b53cf4357845026e4a95cd60f',
 'JUPYTERHUB_API_URL': 'http://127.0.0.1:8081/hub/api',
 'JUPYTERHUB_BASE_URL': '/',
 'JUPYTERHUB_CLIENT_ID': 'user-user6',
 'JUPYTERHUB_HOST': '',
 'JUPYTERHUB_OAUTH_CALLBACK_URL': '/user/user6/oauth_callback',
 'JUPYTERHUB_SERVICE_PREFIX': '/user/user6/',
 'JUPYTERHUB_USER': 'user6',
 'MPLBACKEND': 'module://ipykernel.pylab.backend_inline',
 'PAGER': 'cat',
 'PATH': '/sbin:/bin:/usr/sbin:/usr/bin',
 'SHELL': '/bin/bash',
 'STACKNAME': 'BigData161018',
 'SUBNET': 'subnet-0f774bb671f63cd6a',
 'TERM': 'xterm-color',
 'USER': 'user6',
 'VPCSGID': 'sg-0490f59a9fdd9c28c',
 'WORKSHOP': '161018'}

In [3]:
!aws s3 ls

2017-07-11 14:00:19 aws-athena-query-results-129182954833-eu-west-1
2017-07-11 14:02:39 aws-athena-query-results-129182954833-us-east-1
2017-11-29 15:51:35 aws-athena-query-results-eu-west-1-129182954833
2018-06-06 14:53:10 aws-glue-scripts-129182954833-eu-west-1
2018-06-06 14:53:11 aws-glue-temporary-129182954833-eu-west-1
2018-01-05 12:14:56 billing-oberger-prod1
2017-09-26 15:17:50 cf-templates-15zf4002c55ux-eu-west-1
2018-03-17 10:34:33 cf-templates-15zf4002c55ux-us-east-1
2017-07-10 08:20:49 cloudtrail-awslogs-129182954833-isengard-do-not-delete
2018-08-30 16:25:17 do-not-delete-gatedgarden-audit-129182954833
2018-03-20 15:59:27 sagemaker-eu-west-1-129182954833


In [4]:
!aws s3 ls aws-potus-eu-west-1

                           PRE ML/
                           PRE full/
                           PRE jupyter/
                           PRE sample/
                           PRE split/


**Time to create the bucket that you will use on eu-west-1 on to the workshop**. 

In [5]:
!aws s3api create-bucket --bucket $USER-$WORKSHOP-aws-bigdata-workshop  --region eu-west-1 \
  --create-bucket-configuration LocationConstraint=eu-west-1

{
    "Location": "http://user6-161018-aws-bigdata-workshop.s3.amazonaws.com/"
}


**Now we will copy all data (samples and full dataset)**

In [6]:
!aws s3 sync s3://aws-potus-eu-west-1/ s3://$USER-$WORKSHOP-aws-bigdata-workshop/ \
        --source-region eu-west-1 --region eu-west-1 --only-show-errors

Sample data are now on your bucket

In [7]:
!aws s3 ls s3://$USER-$WORKSHOP-aws-bigdata-workshop/sample/

                           PRE donald-with-sentiment.parquet/
                           PRE last-day-Donald/
                           PRE last-day-Hillary/
                           PRE mini/
                           PRE output/
2018-10-16 08:55:20      39467 AFINN-en-165.txt
2018-10-16 08:55:20      85187 Hillary-450-lines-Sample
2018-10-16 08:55:20      85222 Hillary-450-lines-Sample-Header
2018-10-16 08:55:20  956225022 Hillary-aa
2018-10-16 08:55:20  956225053 Hillary-header
2018-10-16 08:55:58          0 donald-with-sentiment.parquet_$folder$
2018-10-16 08:55:58        228 hillary.q
2018-10-16 08:56:52          0 output_$folder$
2018-10-16 08:56:52         97 select-count-hillary.q


Full data are also on your bucket

In [8]:
!aws s3 ls s3://$USER-$WORKSHOP-aws-bigdata-workshop/split/Hillary/

2018-10-16 08:57:10  956225022 Hillary-aa
2018-10-16 08:57:10  956216347 Hillary-ab
2018-10-16 08:57:10  956261068 Hillary-ac
2018-10-16 08:57:10  956268811 Hillary-ad
2018-10-16 08:57:21  956402067 Hillary-ae
2018-10-16 08:57:29  537251914 Hillary-af


In [9]:
!aws s3 ls s3://$USER-$WORKSHOP-aws-bigdata-workshop/split/Donald/

2018-10-16 08:56:52  940841831 Donald-aa
2018-10-16 08:56:52  941023216 Donald-ab
2018-10-16 08:56:52  941007161 Donald-ac
2018-10-16 08:56:52  941027883 Donald-ad
2018-10-16 08:56:52  940870749 Donald-ae
2018-10-16 08:57:10  538458667 Donald-af


Get the sample locally on an EC2 instance in eu-west-1

In [10]:
!mkdir workshop-data-sample

In [11]:
!aws s3 cp s3://$USER-$WORKSHOP-aws-bigdata-workshop/sample/ workshop-data-sample/ --recursive --only-show-errors

**Have a look inside the data** ; csvkit is a nice tool to play with data at the command-line

In [12]:
!ls workshop-data-sample

AFINN-en-165.txt			hillary.q
Hillary-450-lines-Sample		last-day-Donald
Hillary-450-lines-Sample-Header		last-day-Hillary
Hillary-aa				mini
Hillary-header				output
donald-with-sentiment.parquet		output_$folder$
donald-with-sentiment.parquet_$folder$	select-count-hillary.q


In [13]:
import pandas
df = pandas.read_csv('workshop-data-sample/Hillary-450-lines-Sample-Header', sep='|',error_bad_lines=False)

In [14]:
df.head()

,id,user,message,timestamp,date
0,780543483404808200,williamjones8,RT @HillaryClinton: “You didn’t blame Little J...,1474931042000,2016-09-26 23:04:02
1,780543483258044400,MWChin,Starting with: Are you a perjurer or just a li...,1474931042000,2016-09-26 23:04:02
2,780543482993582100,hassaaninator,RT @HillaryClinton: “You didn’t blame Little J...,1474931042000,2016-09-26 23:04:02
3,780543482486284300,TJ_Pittinger,One of y'all let the 2013 @FSU_Football Team c...,1474931042000,2016-09-26 23:04:02
4,780543482024763400,GabrielleLozano,RT @Jeritron2016: @HillaryClinton has Green pa...,1474931042000,2016-09-26 23:04:02


In [15]:
df.shape

(450, 5)